BRAINSTORM
- IRMAS - predominant instrument recognition
- single instrument: included in part 4 of instrument recog doc

Strategy:

1) create spectograms
2) pass into NN
3) see results and go from there


In [10]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

In [118]:
#what I actually want is two np arrays X and y that preserve mapping of spec to label

#helper function getSpecLab(filename)

import os

def getSpecAndLab(filename):
    y, sr = librosa.load(filename)
    spec = librosa.feature.melspectrogram(y=y)
    label = filename[6:9]
    return spec, label


def createXyFromDir(root):
    specs = []
    labels = []

    percent = 0
    total = len(os.listdir(root)) - 1

    for dir in os.listdir(root):
        d = os.path.join(root, dir).replace("\\","/")
        # checking if it is a dir
        if os.path.isdir(d):
            print(f'Now processing {d}, {(percent/total * 100):.2f}% complete')
            for file in os.listdir(d):
                f = os.path.join(d, file).replace("\\","/")
                spec, label = getSpecAndLab(f)
                specs.append(spec)
                labels.append(label)
        percent+=1
                
    return (np.array(specs), np.array(labels))
    

In [119]:
testspec, testlabel = getSpecAndLab('Audio\cel\[cel][cla]0001__1.wav')

print(f'spec shape: {testspec.shape}, label: {testlabel}')

spec shape: (128, 130), label: cel


In [120]:
root = 'Audio'

X, y = createXyFromDir(root)

Now processing Audio/cel, 0.00% complete
Now processing Audio/cla, 9.09% complete
Now processing Audio/flu, 18.18% complete
Now processing Audio/gac, 27.27% complete
Now processing Audio/gel, 36.36% complete
Now processing Audio/org, 45.45% complete
Now processing Audio/pia, 54.55% complete
Now processing Audio/sax, 72.73% complete
Now processing Audio/tru, 81.82% complete
Now processing Audio/vio, 90.91% complete
Now processing Audio/voi, 100.00% complete


In [121]:
print(X.shape, y.shape)

X = X.reshape(6705, 128, 130, 1)

print(X.shape, y.shape)

print(y[6000])

#now want to get square CNN

(6705, 128, 130) (6705,)
(6705, 128, 130, 1) (6705,)
voi


In [129]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

print(y.shape)

y_transformed = encoder.fit_transform(y.reshape(-1,1)).toarray()

print(y_transformed.shape)

(6705,)
(6705, 11)


In [130]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.1)

In [131]:
# Now test on simple model

import keras
import tensorflow
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Flatten

model = Sequential()

model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(128,130,1)))
model.add(Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Dropout(0.35))

model.add(Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Dropout(0.45))

model.add(Conv2D(256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=2, padding='same'))
model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.75))
model.add(Dense(11, activation='softmax'))


In [132]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

In [133]:
history = model.fit(X_train,
                    y_train,
                    batch_size=32,
                    epochs=75,
                    validation_split=0.1,
                    )

Epoch 1/75
170/170 [==============================] - 263s 2s/step - loss: 2.4442 - accuracy: 0.1600 - val_loss: 2.2324 - val_accuracy: 0.2450
Epoch 2/75
170/170 [==============================] - 249s 1s/step - loss: 2.1709 - accuracy: 0.2483 - val_loss: 2.0307 - val_accuracy: 0.2848
Epoch 3/75
170/170 [==============================] - 248s 1s/step - loss: 2.0519 - accuracy: 0.3002 - val_loss: 1.9626 - val_accuracy: 0.3129
Epoch 4/75
170/170 [==============================] - 248s 1s/step - loss: 1.9681 - accuracy: 0.3344 - val_loss: 1.8942 - val_accuracy: 0.3394
Epoch 5/75
170/170 [==============================] - 247s 1s/step - loss: 1.9012 - accuracy: 0.3630 - val_loss: 1.8480 - val_accuracy: 0.3758
Epoch 6/75
170/170 [==============================] - 247s 1s/step - loss: 1.8581 - accuracy: 0.3707 - val_loss: 1.7351 - val_accuracy: 0.4089
Epoch 7/75
170/170 [==============================] - 246s 1s/step - loss: 1.7890 - accuracy: 0.3948 - val_loss: 1.8519 - val_accuracy: 0.3725